In [ ]:
import os
import numpy as np
from mindspore import Tensor,load_checkpoint,load_param_into_net
from VDSR_net import VDSRNet
import cv2

if __name__ == '__main__':

    model_pth = './pretrained_model.ckpt'
    #VDSR模型
    model = VDSRNet()
    #加载预训练参数
    param_dict = load_checkpoint(model_pth)
    load_param_into_net(model, param_dict)
    #数据集和保存路径
    dataset_path = './Set5'
    save_path = './output'
    if not os.path.exists(save_path):
        # 如果路径不存在，则创建路径
        os.makedirs(save_path)

    for img_name in os.listdir(dataset_path):
        #读取图像，灰度化
        hr_img = cv2.imread(os.path.join(dataset_path,img_name))
        hr_gray_img = cv2.cvtColor(hr_img,cv2.COLOR_BGR2GRAY)
        hr_img_dims = (hr_img.shape[1], hr_img.shape[0])

        #下采样到一半分辨率
        downsampled_gray_img = cv2.resize(hr_gray_img, (0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC)
        #重新上采样到原分辨率（VDSR进行超分辨率计算时图像分辨率不变）
        downsampled_gray_img = cv2.resize(downsampled_gray_img, hr_img_dims, interpolation=cv2.INTER_CUBIC)
        #保存下采样的图像
        cv2.imwrite(os.path.join(save_path,"downsampled_gray_"+img_name), downsampled_gray_img)
        #图像处理
        downsampled_gray_img = np.array(downsampled_gray_img).astype(np.float32)/255  # 将图像数据转换为 numpy 数组
        downsampled_gray_img = np.expand_dims(downsampled_gray_img, axis=0)  # 增加batch_size维度
        downsampled_gray_img = np.expand_dims(downsampled_gray_img, axis=1)  # 添加通道维度
        downsampled_gray_img = Tensor(downsampled_gray_img)

        #网络模型生成超分辨率图像
        sr_gray_img = model(downsampled_gray_img)
        sr_gray_img = (np.array(sr_gray_img)*255).astype(np.int64)
        cv2.imwrite(os.path.join(save_path,"super_revolution_gray_"+img_name), sr_gray_img[0][0])

        #下采样彩色图像
        downsampled_img = cv2.resize(hr_img, (0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC)
        downsampled_img = cv2.resize(downsampled_img, hr_img_dims, interpolation=cv2.INTER_CUBIC)

        cv2.imwrite(os.path.join(save_path,"downsampled_"+img_name), downsampled_img)

        downsampled_img = np.array(downsampled_img).astype(np.float32) / 255  # 将图像数据转换为 numpy 数组
        downsampled_img = np.transpose(downsampled_img, (2, 0, 1))  #通道维度提前

        #VDSR模型第一个卷积层的输入通道为1，针对色彩图像使用逐通道处理的方式
        sr_img = []
        for channel in downsampled_img:
            #网络只能处理单通道图像，因此每个通道单独处理
            channel = np.expand_dims(channel, axis=0)
            channel = np.expand_dims(channel, axis=1)
            channel = Tensor(channel)

            channel = model(channel) * 255
            sr_img.append(channel[0][0])
        sr_img = np.array(sr_img, dtype=np.int64)
        sr_img = np.transpose(sr_img, (1,2,0))
        cv2.imwrite(os.path.join(save_path,"super_revolution_"+img_name), sr_img)


In [ ]:
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import cv2

path = './output'
class_list= ['bird.png','butterfly.png','head.png','woman.png','baby.png']
for class_ in class_list:
    gtimg = cv2.imread('Set5/' + class_)
    gtimg_gray = cv2.cvtColor(gtimg, cv2.COLOR_BGR2GRAY)
    for c_ in ['/downsampled_gray_','/super_revolution_gray_']:
        img1 = cv2.imread(path+c_+class_,0)

        psnr_value = psnr(img1, gtimg_gray)
        ssim_value = ssim(img1, gtimg_gray)
        print(c_ + class_)
        print(psnr_value)
        print(ssim_value)
    for c_ in['/downsampled_','/super_revolution_']:
        img1 = cv2.imread(path + c_ + class_)

        psnr_value = psnr(img1, gtimg)
        ssim_value = ssim(img1, gtimg, multichannel=True)
        print(c_ + class_)
        print(psnr_value)
        print(ssim_value)